In [206]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE

In [467]:
import pandas as pd
import matplotlib.pyplot as plot
import numpy as np
import glob
import os

In [290]:
path = r'C:\\Users\\janik\\Wolke\\OneDrive\\Documents\\Projekte\\waschmaschine\\training'
all_files = glob.glob(path + "\\*.csv")

li = []
i = 0
for filename in all_files:
    i = i + 1
    df = pd.read_csv(filename, index_col=None, header=0, delimiter= ';', decimal= '.')
    li.append(df)

print(i)
data = pd.concat(li, axis=0, ignore_index=True)

26


In [291]:
data

,current,delta,status
0,10.391,0,0
1,9.827,2,0
2,8.800,4,0
3,8.815,6,0
4,8.787,8,0
...,...,...,...
70323,0.045,7306,1
70324,0.045,7308,1
70325,0.045,7310,1
70326,0.045,7312,1


In [292]:
data['current_normalized'] = data['current']/12
data['delta_normalized'] = data['delta']/(120*60)

In [293]:
y = data['status'].to_numpy()

In [489]:
len(y)

70328

In [294]:
x = data[['current_normalized', 'delta_normalized']].to_numpy()

In [233]:
from sklearn.preprocessing import MinMaxScaler

In [476]:
model = LogisticRegression(C=0.1)

In [477]:
model.fit(x, y)

LogisticRegression(C=0.1)

In [393]:
model.classes_

array([0, 1], dtype=int64)

In [488]:
model.intercept_[0]

-5.981861989378207

In [487]:
model.coef_[0][0]

-14.858981685527775

In [486]:
model.coef_[0][1]

9.291278202911599

In [481]:
p_pred = model.predict_proba(x)
y_pred = model.predict(x)
score_ = model.score(x, y)
conf_m = confusion_matrix(y, y_pred)
report = classification_report(y, y_pred)

In [483]:
print('score_:', score_, end='\n\n')

score_: 0.9407206233648049



In [482]:
print('report:', report, sep='\n')

report:
              precision    recall  f1-score   support

           0       0.95      0.98      0.97     62308
           1       0.80      0.64      0.71      8020

    accuracy                           0.94     70328
   macro avg       0.88      0.81      0.84     70328
weighted avg       0.94      0.94      0.94     70328



In [69]:
-5.84409858 - 2.59735626e+01 * 0.045 + 1.72249262e-03 * 4542

0.8106525830400004

In [70]:
1/(1+np.exp(-0.8106525830400004))

0.6922485483553602

In [484]:
def status(current, delta):
    y = model.intercept_[0] + model.coef_[0][0] * (current/12) + model.coef_[0][1] * (delta/7200)
    p = 1/(1+np.exp(-y))
        
    return p

In [485]:
path = r'C:\\Users\\janik\\Wolke\\OneDrive\\Documents\\Projekte\\waschmaschine\\test'
all_files = glob.glob(path + "\\*.csv")

li = []
i = 0
for filename in all_files:
    i = os.path.basename(filename)
    test = pd.read_csv(filename, index_col=None, header=0, delimiter= ';', decimal= '.')
    test['p'] = status(test['current'], test['delta'])
    test.loc[(test['p'] >= 0.5) & (test['current'] < 0.05), 'status_2'] = 1
    test.loc[test['p'] < 0.5, 'status_2'] = 0
    test[(test['status'] == 1) | (test['status_2'] == 1)].to_csv(f'C:\\Users\\janik\\Wolke\\OneDrive\\Documents\\Projekte\\waschmaschine\\validation\\{i}', sep= ';', decimal= '.', index=False)

In [313]:
test = pd.read_csv('C:\\Users\\janik\\Wolke\\OneDrive\\Documents\\Projekte\\waschmaschine\\test\\Waesche_01092021.csv', index_col=None, header=0, delimiter= ';', decimal= '.')

In [338]:
status(0.045/12, 4542/7200)

0.0011147923931813179

In [314]:
test['p'] = status(test['current'], test['delta'])

In [376]:
test[test['p'] >= 0.8] 

,current,delta,status,p,status_2
2162,0.045,4324,1,0.800258,1.0
2163,0.045,4326,1,0.800735,1.0
2164,0.046,4328,1,0.800949,1.0
2165,0.046,4330,1,0.801424,1.0
2166,0.046,4332,1,0.801898,1.0
...,...,...,...,...,...
2511,0.035,5022,1,0.920185,1.0
2512,0.035,5024,1,0.920404,1.0
2513,0.035,5026,1,0.920622,1.0
2514,0.035,5028,1,0.920840,1.0


In [317]:
test

,current,delta,status,p
0,0.035,0,0,0.014559
1,0.035,2,0,0.014581
2,0.035,4,0,0.014603
3,0.035,6,0,0.014625
4,0.067,8,0,0.014460
...,...,...,...,...
2625,0.045,5250,1,0.447755
2626,0.045,5252,1,0.448132
2627,0.045,5254,1,0.448510
2628,0.045,5256,1,0.448888


In [310]:
test.loc[test['p'] >= 0.5, 'status_2'] = 1
test.loc[test['p'] < 0.5, 'status_2'] = 0

In [311]:
test.to_csv('C:\\Users\\janik\\Wolke\\OneDrive\\Documents\\Projekte\\waschmaschine\\validation\\test.csv', sep= ';', decimal= '.', index=False)

In [25]:
model.predict_proba(x)

array([[9.99378183e-01, 6.21816661e-04],
       [9.99995759e-01, 4.24058915e-06],
       [9.99998786e-01, 1.21439796e-06],
       ...,
       [1.61322292e-01, 8.38677708e-01],
       [1.60857503e-01, 8.39142497e-01],
       [1.60393798e-01, 8.39606202e-01]])

In [26]:
model.predict(x)

array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

In [27]:
model.score(x, y)

0.9796893667861409

In [29]:
result.summary()

AttributeError: 'LogisticRegression' object has no attribute 'summary'